### Read in and Combine Daily Data (stored as parquet files)

In [1]:
import os
# from utils.utils import *

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from google.cloud import storage
from pyspark.sql.functions import max, col, count, \
    lit, countDistinct, first
from pyspark.sql.types import FloatType

import matplotlib.pyplot as plt
%matplotlib inline

import time

In [2]:
start_time = time.time()

spark = SparkSession.builder.appName('ParquetExample').getOrCreate()
spark
conf = spark.sparkContext._conf.setAll([('spark.executor.memory', '4g'), ('spark.app.name', 'Spark Updated Conf'), ('spark.executor.cores', '4'), ('spark.cores.max', '4'), ('spark.driver.memory','4g')])
spark.sparkContext.getConf().getAll()

[('spark.stage.maxConsecutiveAttempts', '10'),
 ('spark.dynamicAllocation.minExecutors', '1'),
 ('spark.eventLog.enabled', 'true'),
 ('spark.submit.pyFiles',
  '/root/.ivy2/jars/com.johnsnowlabs.nlp_spark-nlp_2.12-4.4.0.jar,/root/.ivy2/jars/graphframes_graphframes-0.8.2-spark3.1-s_2.12.jar,/root/.ivy2/jars/com.typesafe_config-1.4.2.jar,/root/.ivy2/jars/org.rocksdb_rocksdbjni-6.29.5.jar,/root/.ivy2/jars/com.amazonaws_aws-java-sdk-bundle-1.11.828.jar,/root/.ivy2/jars/com.github.universal-automata_liblevenshtein-3.0.0.jar,/root/.ivy2/jars/com.google.cloud_google-cloud-storage-2.16.0.jar,/root/.ivy2/jars/com.navigamez_greex-1.0.jar,/root/.ivy2/jars/com.johnsnowlabs.nlp_tensorflow-cpu_2.12-0.4.4.jar,/root/.ivy2/jars/it.unimi.dsi_fastutil-7.0.12.jar,/root/.ivy2/jars/org.projectlombok_lombok-1.16.8.jar,/root/.ivy2/jars/com.google.guava_guava-31.1-jre.jar,/root/.ivy2/jars/com.google.guava_failureaccess-1.0.1.jar,/root/.ivy2/jars/com.google.guava_listenablefuture-9999.0-empty-to-avoid-conflict-

### Paths and Data read

In [3]:
defog_metadata = spark.read.csv("gs://msca-bdp-student-gcs/parkinsons_data/defog_metadata.csv", header=True)
subjects = spark.read.csv("gs://msca-bdp-student-gcs/parkinsons_data/subjects.csv", header=True)
daily_metadata = spark.read.csv("gs://msca-bdp-student-gcs/parkinsons_data/daily_metadata.csv", header=True)

In [4]:
daily_path = "parkinsons_data/unlabeled"
top_bucket_name = "msca-bdp-student-gcs"
# full_path = "msca-bdp-student-gcs/parkinsons_data/unlabeled"

Read in 65 parquet files for daily data

In [5]:
# https://cloud.google.com/storage/docs/samples/storage-list-files#storage_list_files-python
# add file name as column - https://sparkbyexamples.com/pyspark/pyspark-lit-add-literal-constant/
# https://www.geeksforgeeks.org/how-to-union-multiple-dataframe-in-pyspark/

client = storage.Client()
blobs = client.list_blobs(top_bucket_name, prefix = daily_path)
num_files_processed = 0

daily_data = None
for i, blob in enumerate(blobs):
    # skip empty blobs
    if blob.size == 0:
        continue
    file_name = blob.name.split("/")[-1].split(".")[0]
    print(f"file {i+1}:", file_name)
    df = spark.read.parquet(f"gs://{top_bucket_name}/{blob.name}")
    df = df.withColumn("Id", lit(file_name))
    if daily_data is None:
        daily_data = df
    else:
        daily_data = daily_data.union(df)
    
    num_files_processed += 1
    if num_files_processed >= 65:
        break
   

file 1: 00c4c9313d


file 2: 07a96f89ec
file 3: 0d1bc672a8
file 4: 0e333c9833
file 5: 164adaed7b
file 6: 17e0c0dc86
file 7: 1c3719ea59
file 8: 1cf80df2d6
file 9: 24016102f2
file 10: 276630050d
file 11: 28e6c306ba
file 12: 2caa348298
file 13: 32bdbc35a0
file 14: 3ae6b0f79f
file 15: 3bd159ded0
file 16: 3f51a63612
file 17: 3fc03f01ed
file 18: 3fe2624b51
file 19: 40bf6c162f
file 20: 418a1ca2c1
file 21: 43ac46d679
file 22: 48081794eb
file 23: 48b636e0f5
file 24: 4b84027351
file 25: 4e44a97a85
file 26: 52fd07ea27
file 27: 5535c94fc9
file 28: 57741bad42
file 29: 5bf570bb7b
file 30: 5e13d48878
file 31: 6e0303484e
file 32: 6ed2f109c3
file 33: 74f1e1e0ba
file 34: 7ab610bb34
file 35: 831c13620e
file 36: 88f67f91db
file 37: 8959244e1c
file 38: 8ca674a988
file 39: 924e997065
file 40: 93abd37fee
file 41: 96f57b4a40
file 42: 9da3e3dc66
file 43: 9fb7805d99
file 44: a213c90b02
file 45: b15168b788
file 46: b18354d4aa
file 47: b1ba59e5f0
file 48: ba970e7e9a
file 49: baac585916
file 50: bbe8b2d4b8
file 51: bd6f65a4d2
file 52:

In [ ]:
daily_data.printSchema()

In [ ]:
daily_data.show(5)

In [ ]:
#display number of records by partition

# def displaypartitions(df):
#     #number of records by partition
#     num = df.rdd.getNumPartitions()
#     print("Partitions:", num)
#     df.withColumn("partitionId", F.spark_partition_id())\
#         .groupBy("partitionId")\
#         .count()\
#         .orderBy(F.asc("count"))\
#         .show(num)
    
# displaypartitions(daily_data)

In [ ]:
# Check number of ids = 65
# num_ids = daily_data.select("Id").distinct().count()
# print(f"Number of unique ids: {num_ids}")


In [ ]:
# look at max time for each person
#max_time = daily_data.select(max("Time")).collect()[0][0]
# max_time

# grouped_data_max = daily_data.groupBy("Id").agg({"Time": "max"})

# for row in grouped_data_max.collect():
#     print(row["Id"], "Max Time:", row["max(Time)"])

In [ ]:
# grouped_data_min = daily_data.groupBy("Id").agg({"Time": "min"})
# for row in grouped_data_min.collect():
#     print(row["Id"], "Min Time:", row["min(Time)"])

In [ ]:
# daily_data.printSchema()

In [6]:
 daily_data = daily_data.withColumn("AccV", col("AccV").cast(FloatType())) \
    .withColumn("AccML", col("AccML").cast(FloatType())) \
    .withColumn("AccAP", col("AccAP").cast(FloatType()))

In [7]:
daily_data.printSchema()

root
 |-- Time: long (nullable = true)
 |-- AccV: float (nullable = true)
 |-- AccML: float (nullable = true)
 |-- AccAP: float (nullable = true)
 |-- Id: string (nullable = false)



## Join with Subject ID

In [ ]:
# daily_metadata.show(1)

In [8]:
daily_data = daily_data.join(daily_metadata, on="Id", how="left")


In [9]:
spark.conf.set("spark.sql.adaptive.enabled", "false")
daily_data.show(2)

+----------+----+----------+------------+----------+-------+-----+------------------------------------+
|        Id|Time|      AccV|       AccML|     AccAP|Subject|Visit|Beginning of recording [00:00-23:59]|
+----------+----+----------+------------+----------+-------+-----+------------------------------------+
|00c4c9313d|   0|  0.328125|   -0.109375|  0.671875| 3d8b73|    1|                               10:19|
|00c4c9313d|   1|0.45310798|-0.124720864|0.81127316| 3d8b73|    1|                               10:19|
+----------+----+----------+------------+----------+-------+-----+------------------------------------+
only showing top 2 rows



In [ ]:
# Check merge - have 65 unique subjects
count_unique_subjects = daily_data.select("Subject").distinct().count()
count_unique_subjects

### Add Time in Seconds Variable

In [10]:
daily_data = daily_data.withColumn("TimeSeconds", col("Time") / 100)
# daily_data.show(5)

In [ ]:
# Check that the conversion makes sense. Most people have ~ 604795 rows. 604795/7 = 86,400 seconds which is expected,
# since there are 86,400 seconds in a day.
# grouped_data_max = daily_data.groupBy("Id").agg({"TimeSeconds": "max"})

# for row in grouped_data_max.collect():
#     print(row["Id"], "Max TimeSeconds:", row["max(TimeSeconds)"])

In [ ]:
# use zander's time function for converstion
# cannot get import of utils to work yet

# daily_hz = 100
# daily_data2 = daily_data.withColumn("Test", lit('null')) \
#     .withColumn("TimeSeconds", convert_time(col("Time").cast(IntegerType()), Hz=daily_hz))

In [ ]:
# daily_data2.select(["Time", "TimeSeconds"]).show(6)

### Reduce Sample Size to Time = 1 Day


In [11]:
# seconds in 1 day = 86400
daily_one_day = daily_data.filter(col("TimeSeconds") <= 86400)
# daily_one_day.show(1)


In [ ]:
# grouped_data_max = daily_one_day.groupBy("Id").agg({"TimeSeconds": "max"})
for row in grouped_data_max.limit(2).collect():
    print(row["Id"], "Max TimeSeconds:", row["max(TimeSeconds)"])


### compare subject ids in daily metadata to the subject ids in defog data

In [20]:
# distinct_defog_subj = defog_metadata.select("Subject")
# distinct_defog_subj.distinct().count()

In [ ]:
# distinct_daily_subj = daily_metadata.select("Subject")
# distinct_daily_subj.distinct().count()

In [ ]:
# combine_defog_daily_meta = distinct_daily_subj.join(distinct_defog_subj, on="Subject", how="inner")
# combine_defog_daily_meta.show(5)
# count = combine_defog_daily_meta.select("Subject").distinct().count()

# subjects_in_both = combine_defog_daily_meta.select("Subject").distinct()

# print(f"{count} subject ids appear in both the daily data and the defog data")


### Check for duplicates, issues, proper number of rows per person

In [19]:
rows = daily_one_day.groupBy('Id').count()
rows.show(65)

+----------+-------+
|        Id|  count|
+----------+-------+
|8ca674a988|8640001|
|164adaed7b|8640001|
|a213c90b02|8640001|
|2caa348298|8640001|
|bbe8b2d4b8|8640001|
|3fe2624b51|8640001|
|d9d96400ea|8640001|
|74f1e1e0ba|8640001|
|c0201855a9|8640001|
|d37397f832|8640001|
|e658b0aa3d|8640001|
|17e0c0dc86|8640001|
|3fc03f01ed|8640001|
|48b636e0f5|8640001|
|3f51a63612|8640001|
|32bdbc35a0|8640001|
|24016102f2|8640001|
|831c13620e|8640001|
|48081794eb|8640001|
|07a96f89ec|8640001|
|ba970e7e9a|8640001|
|924e997065|8640001|
|9fb7805d99|8640001|
|1c3719ea59|8640001|
|b18354d4aa|8640001|
|b1ba59e5f0|8640001|
|4e44a97a85|8640001|
|40bf6c162f|8640001|
|0e333c9833|8640001|
|418a1ca2c1|8640001|
|5bf570bb7b|8640001|
|cb8698473f|8640001|
|e2d103a18e|8640001|
|57741bad42|8640001|
|1cf80df2d6|8640001|
|e328c64434|8640001|
|be948ac8ad|8640001|
|ef1db3ca64|8640001|
|bd6f65a4d2|8640001|
|0d1bc672a8|8640001|
|f16c5cda55|8640001|
|ed0a487f20|8640001|
|93abd37fee|8640001|
|276630050d|8640001|
|52fd07ea27|8

In [13]:
filtered_data = daily_one_day.filter(daily_one_day['Id'] == '74f1e1e0ba')
filtered_data.show(20)

+----------+----+------------+-------------+-------------+-------+-----+------------------------------------+-----------+
|        Id|Time|        AccV|        AccML|        AccAP|Subject|Visit|Beginning of recording [00:00-23:59]|TimeSeconds|
+----------+----+------------+-------------+-------------+-------+-----+------------------------------------+-----------+
|74f1e1e0ba|   0|0.0021972656|-0.0026855469|-0.0043945312| f80d3a|    1|                               08:00|        0.0|
|74f1e1e0ba|   1|0.0024413795| -0.002194287| -0.004882786| f80d3a|    1|                               08:00|       0.01|
|74f1e1e0ba|   2| 0.002197446| -0.001709225| -0.004638792| f80d3a|    1|                               08:00|       0.02|
|74f1e1e0ba|   3|0.0029294614|-0.0026855469|-0.0051265177| f80d3a|    1|                               08:00|       0.03|
|74f1e1e0ba|   4|0.0024354693| -0.002684832| -0.004395484| f80d3a|    1|                               08:00|       0.04|
|74f1e1e0ba|   5|0.00219

In [14]:
end_time = time.time()
total = end_time - start_time
total


197.4713170528412

In [15]:
daily_one_day.show(1)

+----------+----+--------+---------+--------+-------+-----+------------------------------------+-----------+
|        Id|Time|    AccV|    AccML|   AccAP|Subject|Visit|Beginning of recording [00:00-23:59]|TimeSeconds|
+----------+----+--------+---------+--------+-------+-----+------------------------------------+-----------+
|00c4c9313d|   0|0.328125|-0.109375|0.671875| 3d8b73|    1|                               10:19|        0.0|
+----------+----+--------+---------+--------+-------+-----+------------------------------------+-----------+
only showing top 1 row



### Save the data that is filtered to 1 day

In [ ]:
# save the data as a parquet file on gcs
daily_one_day = daily_one_day.withColumnRenamed('Beginning of recording [00:00-23:59]',"record_begin")
daily_one_day.printSchema()
daily_one_day.write.format("parquet").mode("overwrite").save("gs://msca-bdp-student-gcs/parkinsons_data/unlabeled_combined/daily_one_day")


root
 |-- Id: string (nullable = false)
 |-- Time: long (nullable = true)
 |-- AccV: float (nullable = true)
 |-- AccML: float (nullable = true)
 |-- AccAP: float (nullable = true)
 |-- Subject: string (nullable = true)
 |-- Visit: string (nullable = true)
 |-- record_begin: string (nullable = true)
 |-- TimeSeconds: double (nullable = true)



In [18]:
daily_one_day.show(1)

+----------+----+--------+---------+--------+-------+-----+------------+-----------+
|        Id|Time|    AccV|    AccML|   AccAP|Subject|Visit|record_begin|TimeSeconds|
+----------+----+--------+---------+--------+-------+-----+------------+-----------+
|00c4c9313d|   0|0.328125|-0.109375|0.671875| 3d8b73|    1|       10:19|        0.0|
+----------+----+--------+---------+--------+-------+-----+------------+-----------+
only showing top 1 row

